In [3]:
""" Importing bibs """
import requests
from bs4 import BeautifulSoup
import csv
import html5lib

In [2]:
""" Constants """
letters = "abcdefghijklmnopqrstuvwxyz"
outputFile = "./../../data/csv/words.csv"
urlStart = f"https://www.mso.anu.edu.au/%7Eralph/OPTED/v003/wb1913_"
urlEnd = f".html"

In [4]:
""" Functions """


def get_url(letter):
    return urlStart + letter + urlEnd


def row_to_write(words, count):

    for word in words:

        count += 1

        try:
            word_text = word.b.text

        except AttributeError:
            word_text = "a"

        try:
            pos_text = word.i.text

        except AttributeError:
            pos_text = "b"

        meaning_text = (
            word.text.replace(word_text, "", 1)
            .replace(pos_text, "")
            .replace("()", "")
            .strip()
        )

        yield (count, word_text, pos_text, meaning_text)
    return count

In [5]:
""" Main """
with open(outputFile, "w", newline="", encoding="utf-8") as f:

    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)

    writer.writerow(["count", "words", "pos", "meaning"])

    visited = set()
    
    count = 0

    for letter in letters:
        
        url = get_url(letter)

        if url not in visited:

            visited.add(url)

            res = requests.get(url)

            soup = BeautifulSoup(res.text, "html5lib") 

            words = soup.find_all("p")
                                        
            for row in row_to_write(words,count):          
                writer.writerow(row)
                count = row[0]